# **LANDSLIDE** **RISK ZONES ANALYSIS** :
### **A case study of San Cristóbal de Verapaz — Guatemala — Central America.**


---



# **1 : Abstract**


This study analyzes landslide impacts in San Cristóbal de Verapaz, Guatemala, following extreme rainfall events associated with Hurricanes Eta and Iota in late November 5, 2020.

Using Sentinel-2 optical imagery,I generated pre-event and post-event RGB composites to visually assess changes in vegetation and land cover. Spectral indices such as Bare Soil Index (BSI) and Normalized Difference Vegetation Index (NDVI) were computed to identify exposed soil and vegetation loss. Terrain data from SRTM DEM provided slope information, while CHIRPS rainfall data represented triggering conditions. A weighted risk index integrating BSI, slope, NDVI, and rainfall was developed to classify the area into low, medium, and high-risk zones. Additionally, candidate landslide scars were highlighted using combined thresholds for soil exposure, vegetation loss, and steep slopes. The results reveal significant vegetation removal and soil exposure in high-risk zones, primarily on steep slopes in the southern AOI.

This multi-source approach demonstrates the effectiveness of satellite data for landslide monitoring and risk assessment, particularly under climate change scenarios where extreme rainfall events are becoming more frequent in Central America. Challenges remain in threshold selection and resolution mismatches, but the methodology provides a scalable tool for disaster risk reduction and climate resilience planning.



---



# **2 : Introduction**


San Cristóbal de Verapaz, in the mountainous Alta Verapaz region of Guatemala, is highly susceptible to landslides due to steep terrain, fragile soils, and intense seasonal rainfall. The catastrophic rainfall from Hurricanes Eta and Iota in November 2020 triggered widespread landslides and flooding, underscoring the need for effective monitoring and risk assessment in such vulnerable areas. This study applied a multi-source remote sensing approach using Sentinel‑2 Surface Reflectance, Sentinel - 1 GRD, SRTM DEM (USGS/SRTMGL1_003), and CHIRPS Daily Precipitation (UCSB‑CHG/CHIRPS/DAILY) to assess landslide impacts and susceptibility. Visible changes in Sentinel‑2 RGB composites revealed vegetation loss and soil exposure, confirmed by spectral indices such as BSI and NDVI change. A weighted risk index integrating terrain, vegetation, and rainfall factors classified the area into low, medium, and high-risk zones, with high-risk areas concentrated on steep slopes and bare surfaces. Sentinel‑1 backscatter analysis provided additional insights into structural changes, though with limitations related to orbit geometry and moisture sensitivity. Findings demonstrate that satellite-based methods offer cost-effective, scalable tools for landslide monitoring and climate resilience planning. Future improvements include integrating higher-resolution DEMs, soil moisture data, and machine learning for automated early warning systems.



---



# **3: Materials and Methods**
### **Data Used**

We used Sentinel-2 optical imagery, Sentinel-1 radar data, CHIRPS rainfall, and SRTM elevation from the Copernicus and associated Earth observation programs.


*   ### **Sentinel-2 Surface Reflectance -**
 Bands used: B2 (Blue), B3 (Green), B4 (Red), B8 (NIR), B11 (SWIR1), and B12

*   ### **CHIRPS Daily Precipitation (UCSB‑CHG/CHIRPS/DAILY)** - To complement the optical and radar observations, we incorporated CHIRPS rainfall data, which offers quasi-global precipitation estimates at ~5 km resolution. This dataset helped contextualize the hydrometeorological conditions before and after the event.

*   ### **SRTM DEM (USGS/SRTMGL1_003)** - We also used SRTM (Shuttle Radar Topography Mission) elevation data at 30 m resolution to analyze terrain characteristics that may influence flooding, water flow, or landscape response.


*   ## **Sentinel-1 GRD** - supplies radar backscatter measurements in VV polarization using the Interferometric Wide (IW) acquisition mode, with an approximate spatial resolution of 10 m. Radar imagery is particularly valuable because it penetrates clouds and provides consistent surface information independent of atmospheric conditions.

The study area is centered near coordinates (-90.632, 15.391) and is defined by a 10 km × 10 km bounding box.

### **Tools and Libraries**

The analysis was performed using Google Earth Engine (GEE) for cloud-based geospatial processing and Python with the geemap library for visualization and scripting.

### **Principle of the Applied Methods**

The workflow is based on temporal change detection and spectral index analysis. We created a post-event composite image from Sentinel-2 data to visualize the affected area.
To analyse risk zones, several indices were computed:

## BSI = (SWIR + RED) - (NIR + BLUE)/(SWIR + RED) + (NIR + BLUE)

## NDVI = (NIR - RED)/(NIR + RED)

## NDWI = (GREEN - NIR)/(GREEN + NIR)

NDVI (Normalized Difference Vegetation Index) to assess vegetation health.
NDWI (Normalized Difference Water Index) to detect water bodies and sediment-laden water.
BSI (Bare Soil Index) to identify exposed soil, which is common in landslide zones.

A composite image was generated using BSI (Red), NDVI (Green), and NDWI (Blue) to highlight landslide areas in bright yellow/orange tones. Additionally, Sentinel-1 radar data was used to analyze surface changes by comparing backscatter intensity before and after the event. Thresholding (e.g., BSI > 0.3 and NDVI < 0.15) was applied to highlight potential landslide zones.

### **Code Used/Adapted**

The code was adapted from Earth Engine catalogue snippets and extended with scripts for computing BSI, NDVI, NDWI, and creating composite visualizations. Python and geemap library were used to integrate GEE functions, add legends, and display interactive maps. The scripts also include clipping to the area of interest and optional export functions for saving results. AI was prompted to debug and generate codes from my ideas and editing and corrections were done manually.



---



# **4: RESULTS**

## **PART 1 - IMAGE VISUALISATION OF THE LANDSLIDE**

**Initialize Earth Engine: Import & Authenticate**

In [8]:
import ee
ee.Authenticate()

True

In [9]:
ee.Initialize(project="gis-risks") #Initialize EE Session (Project: gis-risks)

**AOI definition (coordinates, point, buffer)**

In [10]:
# ✅ Define coordinates of the area in a variable
coords = [-90.632, 15.391]
landslide_reg = ee.Geometry.Point([-90.632, 15.391])
# ✅ Define date range in a variable
pre_event = ['2020-02-01', '2020-10-30']
post_event = ['2020-12-01', '2021-03-01']

# Composite Image **BEFORE** the Landslide

Images acquired over a span of 10 m0nths before the event.

In [11]:
import geemap
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10   # opaque clouds
    cirrus_bit_mask = 1 << 11  # cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    # Scale surface reflectance to [0,1]
    return image.updateMask(mask).divide(10000)

# -----------------------------
# Region & Dates
# -----------------------------
landslide_reg = ee.Geometry.Point([-90.632, 15.391])

# 5 km buffer, rectangular bounds (as before)
area = landslide_reg.buffer(5000).bounds()

# Pre-event date range (as before)
pre_event = ['2020-02-01', '2020-10-30']

# -----------------------------
# Sentinel-2 SR (pre-event collection)
# -----------------------------
s2_pre = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(pre_event[0], pre_event[1])
    .filterBounds(area)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
    .map(mask_s2_clouds)
)

# Count the number of images (client-side for printing/annotation)
s2_pre_count = s2_pre.size().getInfo()
print(f'Number of Sentinel-2 pre-event images: {s2_pre_count}')

# Build median composite
rgb_before = s2_pre.median().clip(area)

# Visualization parameters (as before)
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# -----------------------------
# Map (display as before)
# -----------------------------
m = geemap.Map()
m.set_center(-90.632, 15.391, 15)

# RGB pre-event layer
m.addLayer(rgb_before, visualization, 'RGB Pre-Event')

# Focus point
m.addLayer(landslide_reg, {'color': 'blue'}, 'Focus Point')

# Red outline for the bounding box (as before, thicker outline)
red_outline = ee.Image().byte().paint(
    featureCollection=ee.FeatureCollection([ee.Feature(area)]),
    color=1,
    width=3
)
m.addLayer(red_outline, {'palette': ['red']}, 'Boundary')


# On-map annotation: black text on white background
m.add_html(f"""
<div style="
  position: fixed;
  top: 20px;
  left: 20px;
  background: #ffffff;        /* white background */
  color: #000000;             /* black text */
  padding: 8px 10px;
  border: 1px solid #000000;  /* black border */
  border-radius: 4px;
  font-size: 13px;
  line-height: 1.4;
  z-index: 9999;">
<b>Sentinel‑2 Pre‑Event Composite</b><br/>
Date range: {pre_event[0]} → {pre_event[1]}<br/>
Images used: <b>{s2_pre_count}</b><br/>
Filter: CLOUDY_PIXEL_PERCENTAGE &lt; 15%
</div>
""")


# Layer control & display
m.addLayerControl()

# Display the map (as before)
m


Number of Sentinel-2 pre-event images: 23


Map(center=[15.391, -90.632], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### **The above shows the area before the landslides hit on 31st November 2020. The area with the blue dot was most affected and we will see the impact on that region in the post event images below.**

### **The number of satellite images considered was 23**

# Composite Image **AFTER** the Landslide



The code below shows the area after the slide. It was mutated from the one above with changes in date alone.
Number of images considered within the 3 month period was 22




In [12]:

def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10   # opaque clouds
    cirrus_bit_mask = 1 << 11  # cirrus
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    # Scale surface reflectance to [0,1]
    return image.updateMask(mask).divide(10000)

# -----------------------------
# Region & Dates
# -----------------------------
landslide_reg = ee.Geometry.Point([-90.632, 15.391])

# 5 km buffer, rectangular bounds
area = landslide_reg.buffer(5000).bounds()

# Post-event date range
post_event = ['2020-11-01', '2021-03-30']

# -----------------------------
# Sentinel-2 SR (post-event collection)
# -----------------------------
s2_post = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(post_event[0], post_event[1])
    .filterBounds(area)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
    .map(mask_s2_clouds)
)

# Count the number of images (client-side for printing/annotation)
s2_post_count = s2_post.size().getInfo()
print(f'Number of Sentinel-2 post-event images: {s2_post_count}')

# Build median composite
rgb_after = s2_post.median().clip(area)

# Visualization parameters
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# -----------------------------
# Map
# -----------------------------
m = geemap.Map()
# Use epicenter coordinates directly instead of undefined `coords`
m.set_center(-90.632, 15.391, 15)

# RGB post-event layer
m.addLayer(rgb_after, visualization, 'RGB Post-Event')

# Epicenter point
m.addLayer(landslide_reg, {'color': 'blue'}, 'Focus Point')

# Red outline for the bounding box
red_outline = ee.Image().byte().paint(
    featureCollection=ee.FeatureCollection([ee.Feature(area)]),
    color=1,
    width=3  # thicker outline
)
m.addLayer(red_outline, {'palette': ['red']}, 'Boundary')

# On-map annotation (black text on white background)
m.add_html(f"""
<div style="
  position: fixed;
  top: 20px;
  left: 20px;
  background: #ffffff;        /* white background */
  color: #000000;             /* black text */
  padding: 8px 10px;
  border: 1px solid #000000;  /* black border */
  border-radius: 4px;
  font-size: 13px;
  line-height: 1.4;
  z-index: 9999;">
<b>Sentinel‑2 Post‑Event Composite</b><br/>
Date range: {post_event[0]} → {post_event[1]}<br/>
Images used: <b>{s2_post_count}</b><br/>
Filter: CLOUDY_PIXEL_PERCENTAGE &lt; 15%
</div>
""")

# Layer control & display
m.addLayerControl()
m


Number of Sentinel-2 post-event images: 22


Map(center=[15.391, -90.632], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### **From the above post landslide satellite image, a disturbance is clearcy seen in the focus point and across some other points scattered around the area of interest. Showing a major lindslide happened in addition to several minor landslide across the hilly regions like the ones on the right and the left**

# **PART 2** - **Quantitative / Exploratory Analysis**



I will be conducting two analysis. These are;

1.   Landslide Risk Zones:
2.   Sentinel‑1 SAR VV Pre/Post & ΔdB Change (Post − Pre):

## **LANDSLIDE RISK ZONES:** San Cristóbal de Verapaz — Guatemala

###**Objective**
###The aim of this analysis was to explore the potential of optical satellite data (Sentinel-2) combined with ancillary datasets (DEM and CHIRPS rainfall) to identify landslide risk zones and highlight areas likely affected by landslides. The focus was on understanding the capabilities, advantages, and limitations of these datasets for landslide assessment.

In [13]:
import ee
import geemap

# Initialize Earth Engine
ee.Initialize(project="gis-risks")

# Define region and date ranges
landslide_reg = ee.Geometry.Point([-90.632, 15.391])
buffer = landslide_reg.buffer(5000)
area = buffer.bounds()

pre_event = ['2020-02-01', '2020-10-30']
post_event = ['2020-11-01', '2021-03-30']

# Function to mask clouds
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

# Sentinel-2 composite (pre-event)
rgb_pre = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(pre_event[0], pre_event[1])
    .filterBounds(area)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
    .map(mask_s2_clouds)
)
rgb_before = (
    rgb_pre.median()
    .clip(area)
)

# Sentinel-2 composite (post-event)
rgb_post = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(post_event[0], post_event[1])
    .filterBounds(area)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
    .map(mask_s2_clouds)
)
rgb_after = (
    rgb_post.median()
    .clip(area)
)

# Get the count of images
rgb_pre_image_count = rgb_pre.size().getInfo()  # Fetch to client
print('Number of pre-event images:', rgb_pre_image_count)

rgb_post_image_count = rgb_post.size().getInfo()  # Fetch to client
print('Number of post-event images:', rgb_post_image_count)


# Compute BSI
bsi = rgb_after.expression(
    '((SWIR + RED) - (NIR + BLUE)) / ((SWIR + RED) + (NIR + BLUE))',
    {
        'SWIR': rgb_after.select('B11'),
        'RED': rgb_after.select('B4'),
        'NIR': rgb_after.select('B8'),
        'BLUE': rgb_after.select('B2')
    }
).rename('BSI')

# Compute NDVI
ndvi = rgb_after.normalizedDifference(['B8', 'B4']).rename('NDVI') #NDVI post to avoid seasonal effects, only to visually confirm scars

ndvi_before = rgb_before.normalizedDifference(['B8', 'B4']).rename('NDVI_Pre')
ndvi_after  = rgb_after.normalizedDifference(['B8', 'B4']).rename('NDVI_Post')
ndvi_change = ndvi_after.subtract(ndvi_before).rename('NDVI_Change')  # range ~[-1, 1]

# DEM for slope
dem = ee.Image('USGS/SRTMGL1_003').clip(area)
slope = ee.Terrain.slope(dem).rename('Slope')

# Rainfall from CHIRPS (monthly mean during post-event period)
rainfall = (
    ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
    .filterDate(pre_event[1], post_event[0])
    .filterBounds(area)
    .mean()
    .clip(area) # Moved .clip(area) here
    .rename('Rainfall')
)

# Normalize layers to 0–1 scale
def normalize(img, min_val, max_val):
    return img.subtract(min_val).divide(max_val - min_val).clamp(0, 1)

bsi_norm = normalize(bsi, -0.3, 1.0)
ndvi_after_norm = normalize(ndvi_after, -1.0, 1.0)
slope_norm = normalize(slope, 0, 60)
rain_norm = normalize(rainfall, 0, 200)  # Adjust max rainfall based on region

# Invert NDVI (low vegetation = high risk)
inv_ndvi_after = ee.Image(1).subtract(ndvi_after_norm)

# Assign weights
w_bsi = 0.4
w_slope = 0.3
w_ndvi_after = 0.2
w_rain = 0.1

# Compute Risk Index
risk_index = (
    bsi_norm.multiply(w_bsi)
    .add(slope_norm.multiply(w_slope))
    .add(inv_ndvi_after.multiply(w_ndvi_after))
    .add(rain_norm.multiply(w_rain))
).rename('Risk_Index')

# ✅ Classify into Low, Medium, High risk zones and clip to AOI
risk_zones = risk_index.expression(
    '(i < 0.1) ? 1 : (i < 0.2) ? 2 : 3',
    {'i': risk_index}
).rename('Risk_Zone').clip(area)

# ✅ Serious Landslide Highlight (BSI > 0.2)
candidate = (bsi.gt(0.2)
             .And(ndvi_change.lt(-0.15))   # vegetation loss
             .And(slope.gt(20)))           # on steeper terrain
serious_landslide = candidate.selfMask()

#serious_landslide = bsi.gt(0.2).selfMask().clip(area)
highlight_vis = {'palette': ['#000000'], 'min': 0, 'max': 1, 'opacity': 1.0}

# Visualization
risk_vis = {'min': 1, 'max': 3, 'palette': ['green', 'yellow', 'red']}  # Low, Medium, High

# Create map
m = geemap.Map()
m.set_center(-90.632, 15.391, 12)

# Add layers
m.addLayer(rgb_before, {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}, 'RGB Pre-Event')
m.addLayer(rgb_after, {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}, 'RGB Post-Event')
m.addLayer(bsi_norm, {'min': 0, 'max': 1, 'palette': ['white', 'brown']}, 'Normalized BSI')
m.addLayer(ndvi_after_norm, {'min': 0, 'max': 1, 'palette': ['brown', 'green']}, 'Normalized NDVI')
m.addLayer(slope_norm, {'min': 0, 'max': 1, 'palette': ['white', 'orange', 'red']}, 'Normalized Slope')
m.addLayer(serious_landslide, highlight_vis, 'Serious Landslide Highlight')
m.addLayer(rain_norm, {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'Normalized Rainfall')
m.addLayer(risk_index, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, 'Risk Index')
m.addLayer(risk_zones, risk_vis, 'Risk Zones (Low-Medium-High)')

# Legend
legend_html = """
<div style="position: fixed; bottom: 30px; left: 30px; background-color: white;
border: 2px solid black; padding: 10px; font-size: 14px;">
<b>Landslide Risk Zones</b><br>
<span style="color:green;">&#9632;</span> Low Risk<br>
<span style="color:yellow;">&#9632;</span> Medium Risk<br>
<span style="color:red;">&#9632;</span> High Risk<br>
</div>
"""
m.add_html(legend_html)

# Enable layer control
m.add_layer_control()

m

Number of pre-event images: 23
Number of post-event images: 22


Map(center=[15.391, -90.632], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### The area covered in red signifies high risk zones. If the DEM layer is viewed, it coincides with areas with steep slopes

*   Risk Zones: Highlights priority areas where multiple landslides trigger factors (rainfall, slope, vegetation) co‑occur.

*   Serious Landslides: A short list of locations for with highest risk.

*   Normalised Rainfall: Light tones = drier conditions, weaker immediate trigger.
Darker blues = wetter soils, stronger potential trigger

*   Normalised BSI: Low values = vegetated surfaces.
High values = bare, bright, or disturbed soil—typical of fresh landslide scars.





### **Sentinel‑1 SAR VV Pre/Post & ΔdB Change (Post − Pre)**

In [17]:

mp = geemap.Map()
mp.set_center(-90.632, 15.391, 12)

def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# -----------------------------
# Sentinel-1 SAR Before & After
# -----------------------------
img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(area)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

sar_before = img_vv.filter(ee.Filter.date(pre_event[0], pre_event[1])).mean().clip(area)
sar_after = img_vv.filter(ee.Filter.date(post_event[0], post_event[1])).mean().clip(area)

sar_vis = {'min': -25, 'max': 5}


# Add SAR pre/post layers
mp.addLayer(sar_before, {'min': -25, 'max': 5}, 'SAR VV Pre-Event (dB)')
mp.addLayer(sar_after,  {'min': -25, 'max': 5}, 'SAR VV Post-Event (dB)')

# --- SAR change in dB: Post - Pre ---
sar_change_dB = sar_after.subtract(sar_before).rename('SAR_Change_dB')
change_vis_dB = {
    'min': -5, 'max': 5,
    'palette': ['#313695','#74add1','#ffffbf','#f46d43','#a50026']  # blue↓ → red↑
}
mp.addLayer(sar_change_dB.clip(area), change_vis_dB, 'S1 VV Change (dB: Post - Pre)')

# Optional: epicenter & AOI outline (if you want them on the SAR map)
mp.addLayer(landslide_reg, {'color': 'blue'}, 'Focus Poiny')
red_outline = ee.Image().byte().paint(
    featureCollection=ee.FeatureCollection([ee.Feature(area)]),
    color=1, width=2
)
mp.addLayer(red_outline, {'palette': ['red']}, 'Boundary')

# -----------------------------
# Legend for SAR Change (ΔdB)
# -----------------------------
legend_html = """
<div style="
  position: fixed;
  bottom: 30px;
  left: 30px;
  background-color: black;
  color: white;
  border: 1px solid white;
  padding: 10px;
  font-size: 13px;
  line-height: 1.4;
  z-index: 9999;
  box-shadow: 0 2px 4px rgba(255,255,255,0.2);
">
  <b>Sentinel‑1 VV Change (ΔdB)</b><br/>
  <span style="display:inline-block;width:14px;height:14px;background:#313695;border:1px solid white;margin-right:6px;"></span> −5 dB (strong decrease)<br/>
  <span style="display:inline-block;width:14px;height:14px;background:#74add1;border:1px solid white;margin-right:6px;"></span> −2 dB<br/>
  <span style="display:inline-block;width:14px;height:14px;background:#ffffbf;border:1px solid white;margin-right:6px;"></span>  0 dB (no change)<br/>
  <span style="display:inline-block;width:14px;height:14px;background:#f46d43;border:1px solid white;margin-right:6px;"></span> +2 dB<br/>
  <span style="display:inline-block;width:14px;height:14px;background:#a50026;border:1px solid white;margin-right:6px;"></span> +5 dB (strong increase)
</div>
"""
mp.add_html(legend_html)

# Layer control and display
mp.add_layer_control()
mp


# Layer control
mp.add_layer_control()

# Display map
mp


Map(center=[15.391, -90.632], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

### **Positive ΔdB (red tones): Increased backscatter—often linked to rougher debris surfaces, higher moisture, or double‑bounce effects on disturbed slopes. These areas are likely impacted by landslide material or wet conditions.**

### **Negative ΔdB (blue tones): Decreased backscatter—can indicate smoother surfaces, radar shadow, ponding, or compaction. These may be areas adjacent to failures or zones of shadowing on steep terrain.**





---



# **5. DISCUSSION**

### **The discussion highlights strengths, limitations, and future improvements for operational landslide monitoring and early warning systems.**

**Key aspects discussed include:**

1.   **Visible satellite changes observed through RGB composites and spectral indices (BSI, NDVI).**
2.  **Risk zone mapping using a weighted index combining terrain, vegetation, and rainfall factors.**
3.   **SAR backscatter analysis for structural changes and its limitations.**









# **1. Visible Satellite changes**

Visible Changes Detected with Sentinel-2

### *   **Pre-event condition:**

The region appeared predominantly green in the RGB composite, indicating dense vegetation cover.


### *   **Post-event condition:**

The post-event composite showed noticeable patches of lighter tones and brownish areas, especially around the landslide epicenter and along steep slopes.


### *   **Interpretation of changes:**
These color changes suggest vegetation removal and soil exposure caused by the landslide.


### *   **Strength of Sentinel-2:**
The high spatial resolution (10 m) allowed these changes to be observed in detail, making Sentinel-2 effective for visual assessment of landslide impacts.


### *   **imitations of this approach:**
Relies on qualitative interpretation of RGB imagery; does not quantify vegetation loss or soil exposure.
Cloud cover during acquisition can limit image availability.
Seasonal variations in vegetation may influence visual interpretation.


# **2: Landslide Risk Zones Using Multi-Source Data**
Data - Sentinel 2, CHIRPS Rainfall, SRTM DEM


### **Integration Approach:**
The landslide risk zones were derived by combining Sentinel-2 (BSI), SRTM DEM (slope), and CHIRPS rainfall. This allowed us to capture both conditioning factors (terrain, vegetation) and triggering factors (rainfall).


### **Slope from SRTM DEM:**

Steep slopes (>20°) were concentrated in the southern part of the AOI.
These areas corresponded closely with high-risk zones, confirming slope as a major driver of landslide susceptibility.
Limitation: SRTM’s 30 m resolution and older acquisition date (2000) reduce its ability to capture fine-scale topographic changes or recent human modifications.



### **Rainfall from CHIRPS:**

Rainfall during the post-event period was significantly higher than the pre-event baseline, supporting its role as a triggering factor.
CHIRPS added valuable climatic context but its coarse spatial resolution (~5 km) cannot capture micro-scale variability in mountainous terrain.



### **Sentinel-2 Spectral Indices (BSI):**

The Bare Soil Index (BSI) is a spectral index designed to highlight areas of exposed soil, which is a key indicator of landslide occurrence and susceptibility. It uses a combination of visible, near-infrared (NIR), and shortwave infrared (SWIR) bands to distinguish bare soil from vegetation and other land cover types.

## **BSI = ((SWIR + RED) - (NIR + BLUE))/((SWIR + RED) + (NIR + BLUE))**

**SWIR = Band 11 (Sentinel‑2)**

**RED = Band 4**

**NIR = Band 8**

**BLUE = Band 2**


**Normalized Difference Vegetation Index (NDVI)**
NDVI is a widely used vegetation index that measures the presence and health of green vegetation by comparing reflectance in the near-infrared (NIR) and red portions of the spectrum.
Formula:

## **NDVI = (NIR - RED)/(NIR + RED)**

Where:

**NIR = Sentinel‑2 Band 88**

**RED = Sentinel‑2 Band 4**

NDVI was calculated for both pre-event and post-event Sentinel‑2 composites to assess vegetation changes caused by the landslide. The NDVI Change was computed as:

NDVI and NDVI Change were critical for validating the risk index and identifying disturbed zones


### **Risk Index Outcome:**

The combined index classified the area into low, medium, and high-risk zones.
High-risk zones were predominantly located on steep slopes with low vegetation cover and high BSI values, validating the integration approach.


## **Critical Assessment**


### **Strengths:**

Multi-source integration captures both physical and climatic drivers of landslides.
Sentinel-2 provides detailed surface information, SRTM adds terrain context, and CHIRPS introduces rainfall triggers.



### **Limitations:**

Resolution mismatch between datasets (10 m vs 30 m vs 5 km) introduces uncertainty.
Sentinel-2 is cloud-dependent, SRTM is static and outdated, and CHIRPS lacks local precision.
For operational early warning, higher-resolution DEM and soil moisture data would enhance accuracy, and Sentinel-1 SAR should be integrated for deformation monitoring.

## **3. SAR Backscatter Change Detection (VV, IW, GRD)**

### **Limitations**

Backscatter is not unique to landslides:
moisture, geometry, and vegetation changes can also drive ΔdB.
Orbit and incidence angle differences (mixing ascending/descending) can introduce change unrelated to the event; ideally keep consistent orbits across pre and post.



---



# **6. Conclusion**


### **This work highlights how satellite data and simple geospatial analytics can provide valuable insights into landslide impacts and risk. By combining Sentinel‑2 imagery for visible changes, terrain information from SRTM, and rainfall data from CHIRPS, we were able to identify areas of vegetation loss, exposed soil, and classify zones into low, medium, and high risk. The approach demonstrates the strength of multi-source integration for capturing both physical and climatic drivers of landslides. Key lessons include the importance of resolution consistency, the role of slope and rainfall as major factors, and the usefulness of spectral indices for detecting surface changes. Future improvements could involve adding radar data (Sentinel‑1) for cloud-independent monitoring, using higher-resolution DEMs and soil moisture data, and applying machine learning or probabilistic models for more accurate and automated risk prediction**



---



# **7. Bibliography**

[Dávila & Ariza (n.d.) : Landslide Detection for Rapid Mapping Using Sentinel-2 Presents Sentinel Hub EvalScript applying BSI along with NDVI, NDWI for quick landslide mapping using Sentinel-2](https://custom-scripts.sentinel-hub.com/custom-scripts/sentinel-2/landslide_detection_rapid_mapping/)

[Ariza et al. (2020) : Landslide Detection in Central America Using the Differential Bare Soil Index
Demonstrates a differential BSI approach in Google Earth Engine for landslide mapping in Mexico](https://www.researchgate.net/publication/352846645_LANDSLIDE_DETECTION_IN_CENTRAL_AMERICA_USING_THE_DIFFERENTIAL_BARE_SOIL_INDEX/fulltext/60dc9a5a92851ca9449b2932/LANDSLIDE-DETECTION-IN-CENTRAL-AMERICA-USING-THE-DIFFERENTIAL-BARE-SOIL-INDEX.pdf)

[Cullen et al. (2022) – Landslide Numerical Factor From CHIRPS Rainfall
Statistical correlation of CHIRPS rainfall with slope and landslide occurrence in Colombia, yielding rain-trigger thresholds.](https://www.mdpi.com/2072-4292/14/9/2239)

[ClimateEngine.org (2021) – Bare Soil Index Description
Defines BSI formula and its application using SWIR, RED, NIR, and BLUE bands](https://support.climateengine.org/article/179-bare-soil-index)

[Jeong et al. (2024) – Landslide Detection Using NDVI and Clustering
Uses NDVI change and ISODATA clustering to detect landslides with 97.7% accuracy](https://www.mdpi.com/2073-445X/14/1/21)

In [19]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:

# Clean the *current* notebook file in Colab and save a copy with "_clean.ipynb".
# If you opened the notebook from Drive, set NOTEBOOK_PATH accordingly.

import os, json, nbformat as nbf

# Try to infer the current filename from Colab env; fallback to manual path.
# If this doesn't resolve, set NOTEBOOK_PATH manually (e.g., '/content/YourNotebook.ipynb').
NOTEBOOK_PATH = /content/drive/MyDrive/Colab Notebooks/Chidiebere's Landslide M1 PROJECT.ipynb
for fn in os.listdir('/content'):
    if fn.endswith('.ipynb'):
        NOTEBOOK_PATH = os.path.join('/content', fn)
        break

if NOTEBOOK_PATH is None:
    raise FileNotFoundError("Couldn't find the notebook file in /content. Set NOTEBOOK_PATH manually.")

CLEAN_PATH = NOTEBOOK_PATH.replace('.ipynb', '_clean.ipynb')

nb = nbf.read(NOTEBOOK_PATH, as_version=nbf.NO_CONVERT)

# Remove notebook-level widgets metadata if present
if 'metadata' in nb and isinstance(nb['metadata'], dict):
    nb['metadata'].pop('widgets', None)

# Remove any cell-level widgets metadata if present
for cell in nb.get('cells', []):
    if isinstance(cell.get('metadata', {}), dict):
        cell['metadata'].pop('widgets', None)

nbf.write(nb, CLEAN_PATH)
print("Wrote cleaned notebook to:", CLEAN_PATH)


FileNotFoundError: Couldn't find the notebook file in /content. Set NOTEBOOK_PATH manually.